### itertools

### Multiple inheritance, method resolution order (mro)

In [ ]:
class A:
    def f(self):
        print("A")

class B:
    def f(self):
        print("B")

class C(B,A):
    pass

C().f()                             
print(C.mro())

In [ ]:
class C(A, B):
    pass

C().f()                             
print(C.mro())

In [50]:
class C(A, B):
    def f(self):
        B.f(self) # in general class.method(self) == object.method()

C().f()

NameError: name 'A' is not defined

### debugging - pdb

### Asyncio

In [ ]:
import asyncio
import time


async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)


async def main():
    task1 = asyncio.create_task(say_after(1, "hello"))

    task2 = asyncio.create_task(say_after(2, "world"))

    print(f"started at {time.strftime('%X')}")

    # Wait until both tasks are completed (should take
    # around 2 seconds.)
    await task1
    await task2

    print(f"finished at {time.strftime('%X')}")

await main()

Normally you would use ```asyncio.run(main())```, jupyter (IPython) is already running an event loop

In [ ]:
import time
from queue import Queue
from threading import Thread


def producer_func(queue):
    print(f"Putting {1}")
    queue.put(1)
    print(f"Putting {2}")
    queue.put(2)
    print(f"Producer waiting for more tasks")
    time.sleep(2)
    print(f"Putting {3}")
    queue.put(3)
    print(f"Producer shuting down")
    queue.put(None)


def consumer_func(queue):
    while True:
        task = queue.get()
        if task is None:
            print(f"Got None, exiting")
            queue.task_done()
            break
        time.sleep(0.5)
        queue.task_done()
        print(f"Task Done {task}")

queue = Queue()
producer = Thread(target=producer_func(queue))
consumer = Thread(target=consumer_func(queue))
producer.start()
consumer.start()
queue.join()

In [ ]:
import asyncio
import time


async def producer(queue):
    print(f"Putting {1}")
    await queue.put(1)
    print(f"Putting {2}")
    await queue.put(2)
    print(f"Producer waiting for more tasks")
    await asyncio.sleep(2)
    print(f"Putting {3}")
    await queue.put(3)
    print(f"Producer shuting down")
    await queue.put(None)


async def consumer(queue):
    while True:
        task = await queue.get()
        if task is None:
            print(f"Got None, exiting")
            queue.task_done()
            break
        time.sleep(0.5)
        queue.task_done()
        print(f"Task Done {task}")


async def main():
    queue = asyncio.queues.Queue()
    producer_coro = asyncio.create_task(producer(queue))
    consumer_coro = asyncio.create_task(consumer(queue))
    await producer_coro
    await consumer_coro
    await queue.join()


await main()


### Itertools

In [ ]:
import itertools

In [ ]:
list(itertools.chain([1, 2], (3, 4, 5), "6"))

In [ ]:
list(itertools.repeat(1,5))

In [ ]:
list(itertools.islice(itertools.count(), 4))

### Enumerate

In [ ]:
values = ["a","b"]

In [ ]:
# DONT DO THIS:
for i in range(len(values)):
    print(f"values[{i}] = {values[i]}")

In [ ]:
for index, value in enumerate(values):
    print(f"values[{index}] = {value}")

In [ ]:
list(enumerate(values)) == [(0, "a"), (1, "b")]

# TODO: dynamic class creation

# TODO: virtualenv, python -m

# TODO: Imports :(

# TODO: dynamic class creation

# TODO: microbenchmarking

### super() TODO

In [ ]:
class Square(Rectangle):
    def __init__(self, length):
        super().__init__(length, length)

class Cube(Square):
    def surface_area(self):
        face_area = super().area()
        return face_area * 6

    def volume(self):
        face_area = super().area()
        return face_area * self.length